# Modelagem 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

# 1. Preparação das Variáveis Alvo (Pedras-Conceito)
def definir_pedra(inde):
    if inde < 6.1: return 'Quartzo'
    elif inde < 7.2: return 'Ágata'
    elif inde < 8.2: return 'Ametista'
    else: return 'Topázio'

df_Base['Pedra_Conceito'] = df_Base['INDE'].apply(definir_pedra)

# 2. Engenharia de Recursos (Features)
# Criando a variável de status de defasagem que vimos ser crítica
df_Base['Status_DEFA'] = df_Base['DEFA'].apply(
    lambda x: 0 if x < 0 else (1 if x == 0 else 2) # 0:Atrasado, 1:No Prazo, 2:Adiantado
)

# Selecionando as colunas preditoras
features = ['IAN', 'IDA', 'IEG', 'IAA', 'IPS', 'IPP', 'IPV', 'FASE', 'Status_DEFA']
X = df_Base[features]
y = df_Base['Pedra_Conceito']

# 3. Tratamento de dados faltantes (Imputação pela mediana)
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# 4. Divisão em Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# 5. Treinamento do Modelo Random Forest
modelo_escola = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_escola.fit(X_train, y_train)

# 6. Avaliação
y_pred = modelo_escola.predict(X_test)
print(f"Acurácia do Modelo: {accuracy_score(y_test, y_pred):.2%}")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

# 7. Identificação de Alunos em Risco para 2025
# Vamos prever para toda a base e filtrar os que o modelo classifica como Quartzo ou Ágata
df_Base['Predicao_2025'] = modelo_escola.predict(X_imputed)
alunos_em_risco = df_Base[df_Base['Predicao_2025'].isin(['Quartzo', 'Ágata'])]

# 8. Importância das Variáveis
importancias = pd.DataFrame({'Feature': features, 'Importancia': modelo_escola.feature_importances_})
print("\nPrincipais Fatores de Decisão do Modelo:")
print(importancias.sort_values(by='Importancia', ascending=False))